## GHG model testing

Try to replicate LiteFarm's ghg testing code

In [1]:
import pandas as pd
import math

In [57]:
df = pd.read_csv("../data/holos_test/test_set.csv")
df

,crop_common_name,farm_id,province,total_area_ha,moisture_%,estimated_yield_kg/ha,N_p,N_s,N_r,N_e,...,tillage,Lignin Content,P_i,PE,FR_Topo,RF_TX,RF_NS,RF_till,RF_CS,RF_AM
0,Soybean,F1,BC,10,14,3500,67.000,6.000,10.00,10.00,...,conventional,0.085,10,20,0.50,1,0.84,1,1,1
1,Soybean,F2,BC,100,14,3500,67.000,6.000,10.00,10.00,...,conventional,0.085,10,20,0.50,1,0.84,1,1,1
2,Soybean,F3,MB,10,14,10000,67.000,6.000,10.00,10.00,...,conventional,0.085,10,20,0.50,1,0.84,1,1,1
3,Soybean,F4,MB,100,14,10000,67.000,6.000,10.00,10.00,...,conventional,0.085,10,20,0.50,1,0.84,1,1,1
4,Soybean,M1,BC,10,14,3500,0.067,0.006,0.01,0.01,...,conventional,0.085,159,678,7.57,1,0.84,1,1,1


In [5]:
df.columns

Index(['crop_common_name', 'farm_id', 'province', 'total_area_ha',
       'moisture_%', 'estimated_yield_kg/ha', 'N_p', 'N_s', 'N_r', 'N_e',
       'R_p', 'R_s', 'R_r', 'R_e', 'lifecycle', 'tillage', 'Lignin Content',
       'P_i', 'PE', 'FR_Topo', 'RF_TX', 'RF_NS', 'RF_till', 'RF_CS', 'RF_AM'],
      dtype='object')

In [86]:
class EmissionCalculator:
    def __init__(self, df):
        """Initialize the EmissionCalculator class."""
        self.df = df
        #print(self.df.columns)
        
    def get_info_for_farm(self, farm_id):
        """Get yield data for a specific farm."""
        return self.df[self.df['farm_id'] == farm_id][['crop_common_name',
                                                    'total_area_ha',
                                                    'estimated_yield_kg/ha',
                                                    'province',
                                                        'farm_id',
                                                        'lifecycle',
                                                        'moisture_%',
                                                        'N_p', 
                                                        'N_s', 
                                                        'R_s', 
                                                        'R_p', 
                                                        'N_r', 
                                                        'N_e', 
                                                        'R_r', 
                                                        'R_e',
                                                        'P_i',
                                                        'PE',
                                                        'FR_Topo',
                                                        'RF_TX',
                                                        'RF_NS',
                                                        'RF_till',
                                                        'RF_CS',
                                                        'RF_AM']]
    
    def calculate_nitrogen_emissions_crop_residue(self, crop_residue, emission_factor):
        """Calculate nitrogen emissions and nitrous oxide produced from crop residue."""
        nitrogen_emission = crop_residue * emission_factor
        conversion_to_nitrous_oxide = nitrogen_emission * 44/28 # convert nitrogen emissions to nitrous oxide
        return conversion_to_nitrous_oxide
    
    def calculate_emission_factor(self,P_i, PE, FR_Topo, RF_TX,	RF_NS, RF_till,	RF_CS, RF_AM):
        """Calculate the emission factor."""
        #P_i = 10; Annual growing season precipitation (May – October), in ecodistrict “i” (mm) (CANSiS atlas)
        #PE = 20;   #Growing season potential evapotranspiration, by ecodistrict (May – October) (CANSiS atlas)

        intermediate_factor = P_i/PE

        # Calculating EF_CT_i: Ecodistrict-level emission factor [kg N2O-N (kg N)-1]
        EF_CT_1 = math.exp(0.00558 * P_i - 7.7) # P_i > PE
        EF_CT_2 = math.exp(0.00558 * PE - 7.7)  # P_i <= PE

        if intermediate_factor > 1:
            # humid environment
            EF_Topo = EF_CT_1

        else:
            # dry environment
            #FR_Topo =  0.1 #10% # Fraction of land occupied by lower portions of landscape

            if P_i == PE:
                #irrigated fields
                EF_Topo = EF_CT_2

            else:
                EF_Topo = ((EF_CT_2*FR_Topo/100)+(EF_CT_1*(1-FR_Topo/100)))

        # at this point we have P_i, PE, FR_Topo, EF_Topo
        EF_base = (EF_Topo * RF_TX)*(1/0.645)

        emission_factor = EF_base * RF_NS * RF_till * RF_CS * RF_AM            

        return emission_factor

    
    def calculate_crop_residue(self, yield_val, moisture, N_p, N_s, R_s, R_p, N_r, N_e, R_r, R_e, area, lifecycle):
        """Calculate crop residue nitrogen for both annual and perennial crops."""

        # Differentiate values based on lifecycle
        if lifecycle == 'ANNUAL':
            S_p = 2  # 2% for annual
            S_s = 100  # Default for annual
        else:  # Perennial
            S_p = 35  # 35% for perennial
            S_s = 100  # Default for perennial

        # Shared calculations
        C_p = ((yield_val * (1 + S_p / 100)) * (1 - moisture / 100)) * 0.45 # Carbon concentration default set to 0.45 by HOLOS V4
        C_ptosoil = C_p * S_p / 100

        # Above-ground residue N calculation
        grain_N = (C_ptosoil / 0.45) * N_p
        C_s = C_p * (R_s / R_p) * (S_s / 100)
        straw_n = (C_s / 0.45) * N_s
        AG_residue_N = grain_N + straw_n

        # Below-ground residue N calculation
        C_r = C_p * (R_r / R_p) * (S_s / 100)
        root_N = (C_r / 0.45) * N_r
        C_e = C_p * (R_e / R_p)
        exudate_n = (C_e / 0.45) * N_e
        BG_residue_N = root_N + exudate_n

        # Total residue N
        total_residue_N = AG_residue_N + BG_residue_N
        
        # Total crop residue
        crop_residue_emissions = total_residue_N * area
        return crop_residue_emissions


    def compute_emissions_for_farm(self, farm_id):
        """Compute the entire emissions calculation process for each crop by specifying a farm_id."""
        farm_data = self.get_info_for_farm(farm_id)
        
        crop_emissions_list = []
        for index, row in farm_data.iterrows():
            crop_name = row['crop_common_name']
            area = row['total_area_ha']
            moisture = row['moisture_%']
            yield_val = row['estimated_yield_kg/ha']
            N_p = row['N_p']
            N_r = row['N_r']
            N_e = row['N_e']
            N_s = row['N_s']
            R_s = row['R_s']
            R_r = row['R_r']
            R_p = row['R_p']
            R_e = row['R_e']
            lifecycle = row['lifecycle'] 
            P_i = row['P_i']
            PE = row['PE']
            FR_Topo = row['FR_Topo']
            RF_TX = row['RF_TX']
            RF_NS = row['RF_NS']
            RF_till = row['RF_till']
            RF_CS = row['RF_CS']
            RF_AM = row['RF_AM']

            # Calculate crop residue
            crop_residue_for_crop = self.calculate_crop_residue(
                yield_val, moisture, N_p, N_s, R_s, R_p, N_r, N_e, R_r, R_e, area, lifecycle
            )
            
            # Calculate emission factor
            emission_factor = self.calculate_emission_factor(
                P_i, PE, FR_Topo, RF_TX, RF_NS, RF_till, RF_CS, RF_AM
            )

            # Calculate nitrogen emissions
            nitrogen_emissions_total = self.calculate_nitrogen_emissions_crop_residue(
                crop_residue_for_crop, emission_factor
            )

            # Ensure required columns have non-null values; otherwise, skip this row
            required_columns = ['total_area_ha','moisture_%', 'estimated_yield_kg/ha', 'N_p', 'N_s', 'R_s', 'R_p', 
                                    'N_r', 'N_e', 'R_r', 'R_e', 'lifecycle','P_i','PE','FR_Topo','RF_TX','RF_NS','RF_till','RF_CS','RF_AM'] 
            if row[required_columns].isnull().any():
                print(f"Warning: Missing data for crop {crop_name} in farm {farm_id}. Skipping emission calculation for this crop.")
                continue
            
            # Store the data in a dictionary
            crop_data = {
                "Farm Name": farm_id,
                "Crop Name": crop_name,
                "Total Area (ha)": area,
                "Province": row['province'],
                "Estimated Yield (kg/ha)": row['estimated_yield_kg/ha'],
                "Emission Factor": emission_factor,
                "Total Crop Residue (N20)": crop_residue_for_crop,
                "Total Nitrogen Emissions from Crop Residue (kg CO2_eq)" : round(nitrogen_emissions_total * 273,1), # convert to carbon 296 default set by HOLOS V4
                "Total Nitrogen Emission from Crop Residue (kg N2O_eq)": round(nitrogen_emissions_total,1),
            }
            crop_emissions_list.append(crop_data)

        return crop_emissions_list
   
    
# ! Find how to calculate the emission factor (P_i, PE, FR_Topo) for each crop. This is according to the location.
# ! Since emission factor is exponential the impact that these have on the final calculation is large, so we want to make sure that these values are consistent and correct
# Note: 
# P_i = 10; Annual growing season precipitation (May – October), in ecodistrict “i” (mm) (CANSiS atlas)
# PE = 20;   #Growing season potential evapotranspiration, by ecodistrict (May – October) (CANSiS atlas)
# Calculating EF_CT_i: Ecodistrict-level emission factor [kg N2O-N (kg N)-1]
#FR_Topo =  0.1 #10% # Fraction of land occupied by lower portions of landscape


In [33]:
class LitefarmEmissionCalculator:
    def __init__(self, df):
        """Initialize the EmissionCalculator class."""
        self.df = df
        #print(self.df.columns)
        
    def get_info_for_farm(self, farm_id):
        """Get yield data for a specific farm."""
        return self.df[self.df['farm_id'] == farm_id][['crop_common_name',
                                                    'total_area_ha',
                                                    'estimated_yield_kg/ha',
                                                    'province',
                                                        'farm_id',
                                                        'lifecycle',
                                                        'moisture_%',
                                                        'N_p', 
                                                        'N_s', 
                                                        'R_s', 
                                                        'R_p', 
                                                        'N_r', 
                                                        'N_e', 
                                                        'R_r', 
                                                        'R_e',
                                                        'P_i',
                                                        'PE',
                                                        'FR_Topo',
                                                        'RF_TX',
                                                        'RF_NS',
                                                        'RF_till',
                                                        'RF_CS',
                                                        'RF_AM']]
    
    def calculate_nitrogen_emissions_crop_residue(self, crop_residue, emission_factor):
        """Calculate nitrogen emissions and nitrous oxide produced from crop residue."""
        nitrogen_emission = crop_residue * emission_factor
        conversion_to_nitrous_oxide = nitrogen_emission * 44/28 # convert nitrogen emissions to nitrous oxide
        return conversion_to_nitrous_oxide
    
    def calculate_emission_factor(self,P_i, PE, FR_Topo, RF_TX,	RF_NS, RF_till,	RF_CS, RF_AM):
        """Calculate the emission factor."""
        #P_i = 10; Annual growing season precipitation (May – October), in ecodistrict “i” (mm) (CANSiS atlas)
        #PE = 20;   #Growing season potential evapotranspiration, by ecodistrict (May – October) (CANSiS atlas)

        intermediate_factor = P_i/PE

        # Calculating EF_CT_i: Ecodistrict-level emission factor [kg N2O-N (kg N)-1]
        EF_CT_1 = math.exp(0.00558 * P_i - 7.7) * 100 # P_i > PE
        EF_CT_2 = math.exp(0.00558 * PE - 7.7) * 100  # P_i <= PE

        if intermediate_factor > 1:
            # humid environment
            EF_Topo = EF_CT_1

        else:
            # dry environment
            #FR_Topo =  0.1 #10% # Fraction of land occupied by lower portions of landscape

            if P_i == PE:
                #irrigated fields
                EF_Topo = EF_CT_2

            else:
                EF_Topo = ((EF_CT_2*FR_Topo)+(EF_CT_1*(1-FR_Topo)))

        # at this point we have P_i, PE, FR_Topo, EF_Topo
        EF_base = (EF_Topo * RF_TX)*(1/0.645)

        emission_factor = EF_base * RF_NS * RF_till * RF_CS * RF_AM            

        return emission_factor

    
    def calculate_crop_residue(self, yield_val, moisture, N_p, N_s, R_s, R_p, N_r, N_e, R_r, R_e, area, lifecycle):
        """Calculate crop residue nitrogen for both annual and perennial crops."""

        # Differentiate values based on lifecycle
        if lifecycle == 'ANNUAL':
            S_p = 2  # 2% for annual
            S_s = 100  # Default for annual
        else:  # Perennial
            S_p = 35  # 35% for perennial
            S_s = 100  # Default for perennial

        # Shared calculations
        C_p = ((yield_val * (1 + S_p / 100)) * (1 - moisture / 100)) * 0.45 # Carbon concentration default set to 0.45 by HOLOS V4
        C_ptosoil = C_p * S_p / 100

        # Above-ground residue N calculation
        grain_N = (C_ptosoil / 0.45) * N_p
        C_s = C_p * (R_s / R_p) * (S_s / 100)
        straw_n = (C_s / 0.45) * N_s
        AG_residue_N = grain_N + straw_n

        # Below-ground residue N calculation
        C_r = C_p * (R_r / R_p) * (S_s / 100)
        root_N = (C_r / 0.45) * N_r
        C_e = C_p * (R_e / R_p)
        exudate_n = (C_e / 0.45) * N_e
        BG_residue_N = root_N + exudate_n

        # Total residue N
        total_residue_N = AG_residue_N + BG_residue_N
        
        # Total crop residue
        crop_residue_emissions = total_residue_N * area
        return crop_residue_emissions


    def compute_emissions_for_farm(self, farm_id):
        """Compute the entire emissions calculation process for each crop by specifying a farm_id."""
        farm_data = self.get_info_for_farm(farm_id)
        
        crop_emissions_list = []
        for index, row in farm_data.iterrows():
            crop_name = row['crop_common_name']
            area = row['total_area_ha']
            moisture = row['moisture_%']
            yield_val = row['estimated_yield_kg/ha']
            N_p = row['N_p']
            N_r = row['N_r']
            N_e = row['N_e']
            N_s = row['N_s']
            R_s = row['R_s']
            R_r = row['R_r']
            R_p = row['R_p']
            R_e = row['R_e']
            lifecycle = row['lifecycle'] 
            P_i = row['P_i']
            PE = row['PE']
            FR_Topo = row['FR_Topo']
            RF_TX = row['RF_TX']
            RF_NS = row['RF_NS']
            RF_till = row['RF_till']
            RF_CS = row['RF_CS']
            RF_AM = row['RF_AM']

            # Calculate crop residue
            crop_residue_for_crop = self.calculate_crop_residue(
                yield_val, moisture, N_p, N_s, R_s, R_p, N_r, N_e, R_r, R_e, area, lifecycle
            )
            
            # Calculate emission factor
            emission_factor = self.calculate_emission_factor(
                P_i, PE, FR_Topo, RF_TX, RF_NS, RF_till, RF_CS, RF_AM
            )

            # Calculate nitrogen emissions
            nitrogen_emissions_total = self.calculate_nitrogen_emissions_crop_residue(
                crop_residue_for_crop, emission_factor
            )

            # Ensure required columns have non-null values; otherwise, skip this row
            required_columns = ['total_area_ha','moisture_%', 'estimated_yield_kg/ha', 'N_p', 'N_s', 'R_s', 'R_p', 
                                    'N_r', 'N_e', 'R_r', 'R_e', 'lifecycle','P_i','PE','FR_Topo','RF_TX','RF_NS','RF_till','RF_CS','RF_AM'] 
            if row[required_columns].isnull().any():
                print(f"Warning: Missing data for crop {crop_name} in farm {farm_id}. Skipping emission calculation for this crop.")
                continue
            
            # Store the data in a dictionary
            crop_data = {
                "Farm Name": farm_id,
                "Crop Name": crop_name,
                "Total Area (ha)": area,
                "Province": row['province'],
                "Estimated Yield (kg/ha)": row['estimated_yield_kg/ha'],
                "Emission Factor": emission_factor,
                "Total Crop Residue (N20)": crop_residue_for_crop,
                "Total Nitrogen Emissions from Crop Residue (kg CO2_eq)" : round(nitrogen_emissions_total * 296,1), # convert to carbon 296 default set by HOLOS V4
                "Total Nitrogen Emission from Crop Residue (kg N2O_eq)": round(nitrogen_emissions_total,1),
            }
            crop_emissions_list.append(crop_data)

        return crop_emissions_list

In [88]:
df = pd.read_csv("../data/holos_test/test_set.csv")
calculator = EmissionCalculator(df)

farm_id = 'M1'  # Replace with the desired farm_id
farm_emissions = calculator.compute_emissions_for_farm(farm_id)

# Display the results
for emission in farm_emissions:
    print("=" * 50)
    for key, value in emission.items():
        print(f"{key}: {value}")
    print("=" * 50)

Farm Name: M1
Crop Name: Soybean
Total Area (ha): 10
Province: BC
Estimated Yield (kg/ha): 3500
Emission Factor: 0.0032860731017619158
Total Crop Residue (N20): 560.2468642105264
Total Nitrogen Emissions from Crop Residue (kg CO2_eq): 789.8
Total Nitrogen Emission from Crop Residue (kg N2O_eq): 2.9


In [87]:
calculator_LF = LitefarmEmissionCalculator(df)

farm_id = 'F1'  # Replace with the desired farm_id
farm_emissions = calculator_LF.compute_emissions_for_farm(farm_id)

# Display the results
for emission in farm_emissions:
    print("=" * 50)
    for key, value in emission.items():
        print(f"{key}: {value}")
    print("=" * 50)

Farm Name: F1
Crop Name: Soybean
Total Area (ha): 10
Province: BC
Estimated Yield (kg/ha): 3500
Emission Factor: 0.06414628604493046
Total Crop Residue (N20): 560246.8642105265
Total Nitrogen Emissions from Crop Residue (kg CO2_eq): 16716190.3
Total Nitrogen Emission from Crop Residue (kg N2O_eq): 56473.6


In [70]:
## poly_id = 551015

df_on_corn =pd.DataFrame({
    'crop_common_name': ['Grain Corn'], 
    'farm_id': ['M2'], 
    'province': ['Ontario'],
    'total_area_ha': [10],
    'moisture_%': [15], 
    'estimated_yield_kg/ha': [8483], 
    'N_p': [0.015], 
    'N_s': [0.005], 
    'N_r': [0.007], 
    'N_e': [0.007],
    'R_p': [0.386], 
    'R_s': [0.387], 
    'R_r': [0.138], 
    'R_e': [0.089], 
    'lifecycle': ['ANNUAL'], 
    'tillage': ['conventional'], 
    'Lignin Content': [0.085],
    'P_i': [445], 
    'PE': [560], 
    'FR_Topo': [6.42], 
    'RF_TX': [1], 
    'RF_NS': [0.84], 
    'RF_till': [1], 
    'RF_CS': [1], 
    'RF_AM': [1]
})

In [68]:
df_on_corn

,crop_common_name,farm_id,province,total_area_ha,moisture_%,estimated_yield_kg/ha,N_p,N_s,N_r,N_e,...,tillage,Lignin Content,P_i,PE,FR_Topo,RF_TX,RF_NS,RF_till,RF_CS,RF_AM
0,Grain Corn,M2,Ontario,10,15,8483,0.015,0.005,0.007,0.007,...,conventional,0.085,445,560,6.42,2.55,0.84,1,1,1


In [71]:
calculator = EmissionCalculator(df_on_corn)

farm_id = 'M2'  # Replace with the desired farm_id
farm_emissions = calculator.compute_emissions_for_farm(farm_id)

# Display the results
for emission in farm_emissions:
    print("=" * 50)
    for key, value in emission.items():
        print(f"{key}: {value}")
    print("=" * 50)

Farm Name: M2
Crop Name: Grain Corn
Total Area (ha): 10
Province: Ontario
Estimated Yield (kg/ha): 8483
Emission Factor: 0.007471990891046478
Total Crop Residue (N20): 693.5196654870465
Total Nitrogen Emissions from Crop Residue (kg CO2_eq): 2410.4
Total Nitrogen Emission from Crop Residue (kg N2O_eq): 8.1


In [74]:
## poly_id_556010
df_poly_id_556010 =pd.DataFrame({
    'crop_common_name': ['Grain Corn'], 
    'farm_id': ['M3'], 
    'province': ['Ontario'],
    'total_area_ha': [10],
    'moisture_%': [15], 
    'estimated_yield_kg/ha': [8033], 
    'N_p': [0.015], 
    'N_s': [0.005], 
    'N_r': [0.007], 
    'N_e': [0.007],
    'R_p': [0.386], 
    'R_s': [0.387], 
    'R_r': [0.138], 
    'R_e': [0.089], 
    'lifecycle': ['ANNUAL'], 
    'tillage': ['conventional'], 
    'Lignin Content': [0.085],
    'P_i': [528], 
    'PE': [541], 
    'FR_Topo': [16.02], 
    'RF_TX': [1], 
    'RF_NS': [0.84], 
    'RF_till': [1], 
    'RF_CS': [1], 
    'RF_AM': [1]
})

In [89]:
calculator = EmissionCalculator(df_poly_id_556010)

farm_id = 'M3'  # Replace with the desired farm_id
farm_emissions = calculator.compute_emissions_for_farm(farm_id)

# Display the results
for emission in farm_emissions:
    print("=" * 50)
    for key, value in emission.items():
        print(f"{key}: {value}")
    print("=" * 50)

Farm Name: M3
Crop Name: Grain Corn
Total Area (ha): 10
Province: Ontario
Estimated Yield (kg/ha): 8033
Emission Factor: 0.011360331284296366
Total Crop Residue (N20): 656.7303398393783
Total Nitrogen Emissions from Crop Residue (kg CO2_eq): 3200.6
Total Nitrogen Emission from Crop Residue (kg N2O_eq): 11.7


In [84]:
## poly_id_556010
df_poly_id_556010_soybean =pd.DataFrame({
    'crop_common_name': ['Soybean'], 
    'farm_id': ['M4'], 
    'province': ['Ontario'],
    'total_area_ha': [10],
    'moisture_%': [15], 
    'estimated_yield_kg/ha': [2241], 
    'N_p': [0.067], 
    'N_s': [0.006], 
    'N_r': [0.01], 
    'N_e': [0.01],
    'R_p': [0.304], 
    'R_s': [0.455], 
    'R_r': [0.146], 
    'R_e': [0.095], 
    'lifecycle': ['ANNUAL'], 
    'tillage': ['conventional'], 
    'Lignin Content': [0.085],
    'P_i': [528], 
    'PE': [541], 
    'FR_Topo': [16.02], 
    'RF_TX': [1], 
    'RF_NS': [0.84], 
    'RF_till': [1], 
    'RF_CS': [1], 
    'RF_AM': [1]
})

In [90]:
calculator = EmissionCalculator(df_poly_id_556010_soybean)

farm_id = 'M4'  # Replace with the desired farm_id
farm_emissions = calculator.compute_emissions_for_farm(farm_id)

# Display the results
for emission in farm_emissions:
    print("=" * 50)
    for key, value in emission.items():
        print(f"{key}: {value}")
    print("=" * 50)

Farm Name: M4
Crop Name: Soybean
Total Area (ha): 10
Province: Ontario
Estimated Yield (kg/ha): 2241
Emission Factor: 0.011360331284296366
Total Crop Residue (N20): 354.5469233526316
Total Nitrogen Emissions from Crop Residue (kg CO2_eq): 1727.9
Total Nitrogen Emission from Crop Residue (kg N2O_eq): 6.3
